# Scaling Dask in Kubernetes

### Table of Contents
1. Dask overview
2. Kubernetes overview
3. Overview of installing Dask
    1. Manually deploying pods and configuring Dask Scheduler/Workers
    2. Installation using Helm
    3. Installation using K8S YAML files
4. Overview  of integrating Dask and Kubernetes
    1. Scaling up/down cluster using dask_kubernetes
        1. Adaptive
        2. Manual
    2. Scaling up/down cluster using kubectl
        1. Auto-Scale
            1. Custom Metrics
        2. Manual
    3.
5. TODOS
    1. How do we create a dask_kubernetes.KubeCluster based on a cluster deployed via helm
    2. How do we configure custom GPU metrics for KubeCluster.adapt()
        1. How do we install custom metric collection for K8s
        2. How do we specify those limits in the dask_kubernetes library?
    3.What are best practices for making storage available across worker pods?


## Dask Overview

Dask is a flexible library for parallel computing in Python. It is purpose-built to parallelize python data science applications from a single laptop all the way up  to a complex 100+ node cluster.

It is composed of a Dask scheduler and a scaling number of Dask workers and the APIs are designed to be familiar for anyone who has used Pandas or Numpy in the past.

By itself Dask accelerates many machine learning applications, but when paired with the additional acclerations of GPUs integrated through the RAPIDS modules it becomes a very powerful tool that no data scientist should be without. 

For more information about dask see [here](https://docs.dask.org/en/latest/)

## Kubernetes
Kubernetes (or K8S) is an open source tool for managing container workloads and services. K8S is designed to scale, and can run on single node systems all the way up to entire clouds.

K8S allows you to deploy docker containers to run tasks. These docker containers are deployed in pods, which can have resources limitations defined, execution commands set, and allows you to specify custom docker images.

K8S allows dynamic resources addition/removal and can be run on-prem, in the cloud, or using hybrid models.

For more information about Kubernetes see [here](https://kubernetes.io/docs/home/)

## Dask and Kubernetes Integration

By combining the cluster-management and auto-scaling capabilities of Kubernetes with the parallel computing and distributed resource management capabilities of dask we can create a data science environment that dynamically determines resources needs, grows to meets those needs, and optimally executes all data processesing, training, and inferencing in our cluster.

## Overview of installing Dask


### Manually deploying pods and configuring Dask Scheduler/Workers


In [ ]:
# TODO:

### Installation using Helm


In [ ]:
# XXX: These must  be run on the K8S managment server, not through this pod.
!helm install -n rapids --namespace rapids --values helm/rapids.yml stable/dask
!kubectl create -f k8s/roles.yaml

### Installation using K8S YAML files

In [ ]:
# TODO:

## Overview of integrating Dask and Kubernetes



### Scaling up/down cluster using dask_kubernetes


#### Adaptive

In [ ]:
import dask_kubernetes as dk
from dask.distributed import Client

client = Client()
client
cluster.adapt()

In [ ]:
# TODO: Run workload

In [ ]:
client

#### Manual

In [ ]:
import dask_kubernetes as dk
from dask.distributed import Client

client = Client()
client

In [ ]:
cluster = dk.KubeCluster.from_yaml('/rapids/notebooks/worker-spec-dynamic.yaml')
cluster.scale(3)

In [ ]:
client

### Scaling up/down cluster using kubectl


#### Auto-Scale
##### Custom Metrics

In [8]:
# TODO: show custom metrics

In [ ]:
!kubectl create -f k8s/auto-scale.yaml # TODO: This file isn't in the Dockerfile yet

#### Manual

In [ ]:
!kubectl -n rapids scale deployment rapids-dask-worker --replicas=0 # TODO: This may not work in this Dockerfile yet

In [ ]:
!kubectl -n rapids scale deployment rapids-dask-worker --replicas=3 # TODO: This may not work in this Dockerfile yet

## TODOS
1. How do we create a dask_kubernetes.KubeCluster based on a cluster deployed via helm
2. How do we configure custom GPU metrics for KubeCluster.adapt()
3. How do we install custom metric collection for K8s
4. How do we specify those limits in the dask_kubernetes library? 3.What are best practices for making storage available across worker pods?